## Imports, function definitions

Note: DeepMoD does not work for python3.9

In [1]:
# pip install pysindy

In [2]:
# pip install DeePyMoD

In [3]:
# pip install --upgrade tensorflow

In [4]:
# pip install tensorflow-io

In [5]:
results_path = ""

In [6]:
import matplotlib.pyplot as plt

# General imports
import numpy as np
import torch
import pandas as pd

# DeePyMoD imports
from deepymod import DeepMoD
from deepymod.data import Dataset, get_train_test_loader
from deepymod.data.samples import Subsample_random
from deepymod.data.burgers import burgers_delta
from deepymod.model.constraint import LeastSquares
from deepymod.model.func_approx import NN
from deepymod.model.library import Library1D
from deepymod.model.sparse_estimators import Threshold
from deepymod.training import train
from deepymod.training.sparsity_scheduler import Periodic, TrainTest, TrainTestPeriodic

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(device)

# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


import pickle
from sklearn.metrics import mean_squared_error
def load_data(X,y):
    return torch.from_numpy(X).float(),torch.from_numpy(y).float()

def uxt_2D_to1D(u,x,t):
    '''
    Function for reshaping u,x and t
    '''
    t_reg=[]
    x_reg=[]
    for i in range(len(t)):
        for j in range(len(x)):
            t_reg.append(t[i])
            x_reg.append(x[j])
    u_reg = u.reshape((u.size, 1))

    return u_reg,x_reg,t_reg

descr = ['',
 'u_{x}',
 'u_{xx}',
 'u_{xxx}',
 'u',
 'uu_{x}',
 'uu_{xx}',
 'uu_{xxx}',
 'u^2',
 'u^2u_{x}',
 'u^2u_{xx}',
 'u^2u_{xxx}']

def return_pde(w, rhs_description=descr, ut = 'u_t'):
    pde = ut + ' = '
    first = True
    for i in range(len(w)):
        if w[i] != 0:
            if not first:
                pde = pde + ' + '
            pde = pde + "(%05f)" % (w[i].real) + rhs_description[i] 
            first = False
    return pde


cuda


In [7]:
def error_metric(true_coeff,pred_coeff):
    wrmse = mean_squared_error(true_coeff,pred_coeff,squared=False)
    return wrmse

## Testing 2.1

In [8]:
# Heat equation ut = 0.15uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.15

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("1_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(11,13):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))
    

SNR =  300
Dataset is using device:  cuda


2023-04-28 11:01:32.509266: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 11:01:33.049061: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


 19750  MSE: 7.34e+03  Reg: 1.59e-11  L1: 2.30e+07 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-1.147457) + (-1201060.250000)u_{x} + (29640.351562)u_{xx} + (143309.171875)u_{xxx} + (0.039522)u + (-1976.188477)uu_{x} + (1078.259521)uu_{xx} + (-50.150208)uu_{xxx} + (-0.000124)u^2 + (21.510952)u^2u_{x} + (-4.122305)u^2u_{xx} + (-1.569126)u^2u_{xxx}  rmse =  349281.0300442485
Dataset is using device:  cuda
 19750  MSE: 1.57e+04  Reg: 6.88e-12  L1: 1.15e+07 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-14.616911) + (4278899.500000)u_{x} + (6615563.500000)u_{xx} + (-1006549.812500)u_{xxx} + (0.203274)u + (-10470.327148)uu_{x} + (-34086.843750)uu_{xx} + (7744.185059)uu_{xxx} + (-0.000610)u^2 + (-36.172653)u^2u_{x} + (22.433672)u^2u_{xx} + (-14.611698)u^2u_{xxx}  rmse =  2292908.14995055
Dataset is using device:  cuda
 19750  MSE: 1.64e+04  Reg: 3.58e-12  L1: 1.44e+07 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.256954) + (-1

KeyboardInterrupt: 

In [8]:
# Heat equation ut = 0.15uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.15

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("1_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(12,13):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))
    

SNR =  200
Dataset is using device:  cuda


2023-04-30 12:08:17.641237: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-30 12:08:18.265894: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


  2250  MSE: 1.07e+05  Reg: 4.53e-04  L1: 9.34e+05 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000000) + (-257426.812500)u_{x} + (-9587.468750)u_{xx} + (-6548.296875)u_{xxx} + (49825.875000)uu_{x} + (-1564.250000)uu_{xx} + (1980.179688)uu_{xxx} + (-2237.423828)u^2u_{x} + (334.631348)u^2u_{xx} + (-141.758301)u^2u_{xxx}  rmse =  75772.40612918999
Dataset is using device:  cuda
  2250  MSE: 1.07e+05  Reg: 2.69e-07  L1: 4.38e+05 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (62354.179688)u_{x} + (-51149.324219)u_{xxx} + (-6830.488281)uu_{x} + (-14923.336914)uu_{xx} + (6407.210938)uu_{xxx} + (1634.205444)u^2u_{xx} + (-88.014008)u^2u_{xxx}  rmse =  23835.17948549074
Dataset is using device:  cuda
  2000  MSE: 1.05e+05  Reg: 9.90e-14  L1: 9.90e-01 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.266470)u_{x} + (-0.172165)u_{xx}  rmse =  0.12069111603711709
Dataset is using device:  cuda
  2500  MSE: 1.06e+05  Reg: 1.74e-06  L1: 

## Testing 2.2

In [9]:
# Heat equation ut = 0.15uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.15

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("1_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(13,15):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))
    

SNR =  100
Dataset is using device:  cuda
  2250  MSE: 1.06e+05  Reg: 1.21e-05  L1: 2.56e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000001) + (-579030.125000)u_{x} + (-792025.687500)u_{xx} + (66163.234375)uu_{x} + (140376.421875)uu_{xx} + (-635.806641)u^2u_{x} + (-6041.633789)u^2u_{xx}  rmse =  286749.140061978
Dataset is using device:  cuda
  2250  MSE: 1.05e+05  Reg: 5.72e-07  L1: 1.41e+05 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000012) + (-0.129139)u_{xx} + (-18628.095703)u_{xxx} + (5946.857422)uu_{x} + (2017.795044)uu_{xxx} + (-644.463806)u^2u_{x}  rmse =  5677.8655998214035
Dataset is using device:  cuda
  2250  MSE: 1.07e+05  Reg: 4.75e-03  L1: 2.56e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000016) + (244596.046875)u_{xx} + (-654.053894)u_{xxx} + (-4678.242676)uu_{x} + (-60282.785156)uu_{xx} + (-272.637817)uu_{xxx} + (550.541016)u^2u_{x} + (3710.377930)u^2u_{xx} + (41.175289)u^2u_{xxx}  rmse

## Testing 2.3

In [10]:


# Heat equation ut = 0.15uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.15

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("1_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(15,17):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))
    

SNR =  80
Dataset is using device:  cuda
 19750  MSE: 1.06e+05  Reg: 2.48e-07  L1: 3.36e+05 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.000010) + (-0.166853)u_{x} + (-11044.797852)u_{xxx} + (-6959.268555)uu_{xx} + (-561.840454)uu_{xxx} + (749.419312)u^2u_{xx} + (188.422272)u^2u_{xxx}  rmse =  3778.5775035202764
Dataset is using device:  cuda
  1500  MSE: 1.07e+05  Reg: 4.64e-13  L1: 9.68e-01 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.312190)u_{xx} + (0.033356)u_{xxx}  rmse =  0.13376976976354538
Dataset is using device:  cuda
  2500  MSE: 1.05e+05  Reg: 7.06e-06  L1: 1.40e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (29434.097656)u_{xx} + (-6130.630859)u_{xxx} + (-16299.468750)uu_{x} + (-5097.102051)uu_{xx} + (1303.377075)uu_{xxx} + (1757.313965)u^2u_{x} + (220.668274)u^2u_{xx} + (-70.104843)u^2u_{xxx}  rmse =  10001.814808491921
Dataset is using device:  cuda
  2250  MSE: 1.06e+05  Reg: 2.93e-04  L1: 9.24e+0

## Testing 2.4

In [11]:


# Heat equation ut = 0.15uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.15

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("1_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(17,19):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))
    

SNR =  60
Dataset is using device:  cuda
  2000  MSE: 1.06e+05  Reg: 3.58e-13  L1: 9.42e-01 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.141861)u_{xx} + (0.035390)u_{xxx}  rmse =  0.08487025415381116
Dataset is using device:  cuda
 19750  MSE: 1.06e+05  Reg: 1.70e-05  L1: 8.06e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000001) + (1612078.125000)u_{xx} + (217784.484375)u_{xxx} + (227876.718750)uu_{x} + (-111871.328125)uu_{xx} + (-19113.308594)uu_{xxx} + (-22408.621094)u^2u_{x} + (-4579.574707)u^2u_{xx} + (-230.037598)u^2u_{xxx}  rmse =  475355.740683212
Dataset is using device:  cuda
  2500  MSE: 1.06e+05  Reg: 1.26e-05  L1: 8.65e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (693952.500000)u_{xx} + (58007.343750)u_{xxx} + (129256.843750)uu_{x} + (-77143.000000)uu_{xx} + (-954.401978)uu_{xxx} + (-15743.427734)u^2u_{x} + (-894.395264)u^2u_{xx} + (-747.296387)u^2u_{xxx}  rmse =  205718.90508235677
Dataset is usi

## Testing 2.5

In [8]:


# Heat equation ut = 0.15uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.15

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("1_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(19,21):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))
    

SNR =  40
Dataset is using device:  cuda


2023-05-05 16:10:27.294707: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-05 16:10:27.870648: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


  2250  MSE: 1.06e+05  Reg: 2.30e-04  L1: 2.53e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000002) + (68156.851562)u_{xx} + (-33503.316406)u_{xxx} + (-441.466919)uu_{x} + (-14150.460938)uu_{xx} + (8297.829102)uu_{xxx} + (56.386574)u^2u_{x} + (717.107056)u^2u_{xx} + (-516.118408)u^2u_{xxx}  rmse =  22431.147213688975
Dataset is using device:  cuda
  2250  MSE: 1.08e+05  Reg: 7.26e-07  L1: 3.71e+04 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000024) + (-27308.990234)u_{x} + (-0.616564)u_{xx} + (0.317653)u_{xxx} + (3381.578857)uu_{x}  rmse =  7943.634893625397
Dataset is using device:  cuda
  1750  MSE: 1.06e+05  Reg: 9.06e-11  L1: 9.35e-01 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-1.929895)u_{x}  rmse =  0.5587928237090635
Dataset is using device:  cuda
  2250  MSE: 1.07e+05  Reg: 1.40e-02  L1: 1.57e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (139668.515625)u_{x} + (-108113.062500)u_